In [1]:
import numpy as np
import optuna
from optuna.integration import CatBoostPruningCallback
import logging
import sys

import catboost as cb
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, log_loss, make_scorer, classification_report
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

import warnings
from optuna.exceptions import ExperimentalWarning

warnings.filterwarnings("ignore", category=ExperimentalWarning)

In [2]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "Test10" 
storage_name = "sqlite:///LitmusOptunaTest.sqlite"

In [3]:
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), 
    direction="minimize",
    study_name=study_name, 
    storage=storage_name,
    load_if_exists=True
    )

[I 2023-04-13 21:06:36,296] Using an existing study with name 'Test10' instead of creating a new one.


Using an existing study with name 'Test10' instead of creating a new one.


In [11]:
X, y = make_classification(
    n_samples=5000, # 1000 observations 
    n_features=500, # 5 total features
    n_informative=100, # 3 'useful' features
    n_classes=2, # binary target/label 
    random_state=999 # if you want the same results as mine
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [12]:
y_train = np.where(y_train == 1, "yes", "no")
y_test = np.where(y_test == 1, "yes", "no")

In [13]:
# Default params
clf2 = cb.CatBoostClassifier(verbose=False)
clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

          no       0.91      0.91      0.91       630
         yes       0.91      0.91      0.91       620

    accuracy                           0.91      1250
   macro avg       0.91      0.91      0.91      1250
weighted avg       0.91      0.91      0.91      1250



In [14]:
# OptunaSearchCV approach
clf = cb.CatBoostClassifier(verbose=False)

param_distributions = {
    "learning_rate": optuna.distributions.FloatDistribution(1e-3, 1.0, log=True),
    "l2_leaf_reg": optuna.distributions.FloatDistribution(1e-8, 100.0, log=True),
    "colsample_bylevel": optuna.distributions.FloatDistribution(0.01, 0.1, log=True),
    "depth": optuna.distributions.IntDistribution(1, 12),
    "random_strength": optuna.distributions.FloatDistribution(1e-8, 10.0, log=True),
    "bootstrap_type": optuna.distributions.CategoricalDistribution(["Bayesian", "Bernoulli", "MVS"]),
    "grow_policy": optuna.distributions.CategoricalDistribution(["SymmetricTree", "Depthwise", "Lossguide"]),
    "od_type": optuna.distributions.CategoricalDistribution(["IncToDec", "Iter"]),
    "od_wait": optuna.distributions.IntDistribution(10, 50)
    }

optuna_search = optuna.integration.OptunaSearchCV(
    clf, param_distributions, cv=3, enable_pruning=False, 
    study=study, n_trials=5, verbose=False,
    scoring=make_scorer(log_loss, greater_is_better=False, needs_proba=True))

optuna_search.fit(X_train, y_train)
y_pred = optuna_search.predict(X_test)

print(classification_report(y_test, y_pred))

[I 2023-04-11 09:28:37,147] Trial 39 finished with value: -0.6511535966073816 and parameters: {'learning_rate': 0.0010671445753259938, 'l2_leaf_reg': 93.70655010457759, 'colsample_bylevel': 0.024582970703969854, 'depth': 11, 'random_strength': 3.415251558759658e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 45}. Best is trial 38 with value: 0.6440391395231012.


Trial 39 finished with value: -0.6511535966073816 and parameters: {'learning_rate': 0.0010671445753259938, 'l2_leaf_reg': 93.70655010457759, 'colsample_bylevel': 0.024582970703969854, 'depth': 11, 'random_strength': 3.415251558759658e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 45}. Best is trial 38 with value: 0.6440391395231012.
Trial 39 finished with value: -0.6511535966073816 and parameters: {'learning_rate': 0.0010671445753259938, 'l2_leaf_reg': 93.70655010457759, 'colsample_bylevel': 0.024582970703969854, 'depth': 11, 'random_strength': 3.415251558759658e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 45}. Best is trial 38 with value: 0.6440391395231012.


[I 2023-04-11 09:29:05,643] Trial 40 finished with value: -0.5992858517784798 and parameters: {'learning_rate': 0.002711761957783622, 'l2_leaf_reg': 78.86480925339944, 'colsample_bylevel': 0.025271655034679888, 'depth': 9, 'random_strength': 4.572303601353441e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 44}. Best is trial 38 with value: 0.6440391395231012.


Trial 40 finished with value: -0.5992858517784798 and parameters: {'learning_rate': 0.002711761957783622, 'l2_leaf_reg': 78.86480925339944, 'colsample_bylevel': 0.025271655034679888, 'depth': 9, 'random_strength': 4.572303601353441e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 44}. Best is trial 38 with value: 0.6440391395231012.
Trial 40 finished with value: -0.5992858517784798 and parameters: {'learning_rate': 0.002711761957783622, 'l2_leaf_reg': 78.86480925339944, 'colsample_bylevel': 0.025271655034679888, 'depth': 9, 'random_strength': 4.572303601353441e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 44}. Best is trial 38 with value: 0.6440391395231012.


[I 2023-04-11 09:29:37,223] Trial 41 finished with value: -0.637930286596304 and parameters: {'learning_rate': 0.0010213923766462843, 'l2_leaf_reg': 5.856320541755137, 'colsample_bylevel': 0.035782457763539645, 'depth': 12, 'random_strength': 3.651032606216074e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 46}. Best is trial 38 with value: 0.6440391395231012.


...

Trial 56 finished with value: -0.4271223320841751 and parameters: {'learning_rate': 0.016433998471879978, 'l2_leaf_reg': 9.393690785460196, 'colsample_bylevel': 0.02268849283921295, 'depth': 10, 'random_strength': 2.5855076581897606e-07, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 48}. Best is trial 38 with value: 0.6440391395231012.
Trial 56 finished with value: -0.4271223320841751 and parameters: {'learning_rate': 0.016433998471879978, 'l2_leaf_reg': 9.393690785460196, 'colsample_bylevel': 0.02268849283921295, 'depth': 10, 'random_strength': 2.5855076581897606e-07, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 48}. Best is trial 38 with value: 0.6440391395231012.


[I 2023-04-11 09:36:35,104] Trial 57 finished with value: -0.5716107138416362 and parameters: {'learning_rate': 0.003580102242672655, 'l2_leaf_reg': 98.92411394882376, 'colsample_bylevel': 0.030984279213829395, 'depth': 11, 'random_strength': 1.8237032934954446e-06, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'IncToDec', 'od_wait': 50}. Best is trial 38 with value: 0.6440391395231012.


Trial 57 finished with value: -0.5716107138416362 and parameters: {'learning_rate': 0.003580102242672655, 'l2_leaf_reg': 98.92411394882376, 'colsample_bylevel': 0.030984279213829395, 'depth': 11, 'random_strength': 1.8237032934954446e-06, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'IncToDec', 'od_wait': 50}. Best is trial 38 with value: 0.6440391395231012.
Trial 57 finished with value: -0.5716107138416362 and parameters: {'learning_rate': 0.003580102242672655, 'l2_leaf_reg': 98.92411394882376, 'colsample_bylevel': 0.030984279213829395, 'depth': 11, 'random_strength': 1.8237032934954446e-06, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'IncToDec', 'od_wait': 50}. Best is trial 38 with value: 0.6440391395231012.


[I 2023-04-11 09:36:59,575] Trial 58 finished with value: -0.6441049458705317 and parameters: {'learning_rate': 0.0012847232065872172, 'l2_leaf_reg': 2.315414653533972, 'colsample_bylevel': 0.018229022536633683, 'depth': 10, 'random_strength': 7.652287494347732e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 46}. Best is trial 38 with value: 0.6440391395231012.


Trial 58 finished with value: -0.6441049458705317 and parameters: {'learning_rate': 0.0012847232065872172, 'l2_leaf_reg': 2.315414653533972, 'colsample_bylevel': 0.018229022536633683, 'depth': 10, 'random_strength': 7.652287494347732e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 46}. Best is trial 38 with value: 0.6440391395231012.
Trial 58 finished with value: -0.6441049458705317 and parameters: {'learning_rate': 0.0012847232065872172, 'l2_leaf_reg': 2.315414653533972, 'colsample_bylevel': 0.018229022536633683, 'depth': 10, 'random_strength': 7.652287494347732e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 46}. Best is trial 38 with value: 0.6440391395231012.
              precision    recall  f1-score   support

          no       0.84      0.84      0.84       630
         yes       0.84      0.84      0.84       620

    accuracy                           0.84      1250
   macro avg       0

In [6]:
# CV with pruning approach

ho_objective = lambda trial: get_ho_objective(trial, X_train, y_train, X_test, y_test)

def get_ho_objective(trial: optuna.Trial, X_train, y_train, X_test, y_test) -> float:

    params = {
        "objective": "Logloss",
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1.0, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "random_strength": trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "grow_policy": trial.suggest_categorical(
            "grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        "od_type": trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        "od_wait": trial.suggest_int("od_wait", 10, 50),
        "eval_metric": "Logloss"
    }
    
    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    gbm = cb.CatBoostClassifier(**params)

    pruning_callback = CatBoostPruningCallback(trial, "Logloss")
    
    gbm.fit(
        X_train,
        y_train,
        eval_set=[(X_test, y_test)],
        verbose=0,
        early_stopping_rounds=10,
        callbacks=[pruning_callback]
    )

    # evoke pruning manually.
    pruning_callback.check_pruned()

    pred_proba = gbm.predict_proba(X_test)

    return log_loss(y_test, pred_proba)

In [7]:
study.optimize(ho_objective, n_trials=10, timeout=600)

[I 2023-04-06 17:19:46,107] Trial 29 finished with value: 0.4965486872788487 and parameters: {'learning_rate': 0.0018769895006530033, 'l2_leaf_reg': 1.1209179574393942, 'colsample_bylevel': 0.0492000624281411, 'depth': 9, 'random_strength': 7.913712469446155e-07, 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise', 'od_type': 'Iter', 'od_wait': 39}. Best is trial 27 with value: 0.4966336815104927.


Trial 29 finished with value: 0.4965486872788487 and parameters: {'learning_rate': 0.0018769895006530033, 'l2_leaf_reg': 1.1209179574393942, 'colsample_bylevel': 0.0492000624281411, 'depth': 9, 'random_strength': 7.913712469446155e-07, 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise', 'od_type': 'Iter', 'od_wait': 39}. Best is trial 27 with value: 0.4966336815104927.


[I 2023-04-06 17:21:40,777] Trial 30 finished with value: 0.4963161307961581 and parameters: {'learning_rate': 0.0016743316892679659, 'l2_leaf_reg': 0.7515248848732622, 'colsample_bylevel': 0.04308438163996671, 'depth': 12, 'random_strength': 4.4941525596606077e-07, 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise', 'od_type': 'Iter', 'od_wait': 39}. Best is trial 27 with value: 0.4966336815104927.


Trial 30 finished with value: 0.4963161307961581 and parameters: {'learning_rate': 0.0016743316892679659, 'l2_leaf_reg': 0.7515248848732622, 'colsample_bylevel': 0.04308438163996671, 'depth': 12, 'random_strength': 4.4941525596606077e-07, 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise', 'od_type': 'Iter', 'od_wait': 39}. Best is trial 27 with value: 0.4966336815104927.


[I 2023-04-06 17:23:29,507] Trial 31 finished with value: 0.4843430139592682 and parameters: {'learning_rate': 0.0018047419737989524, 'l2_leaf_reg': 0.5658974760505915, 'colsample_bylevel': 0.04315931488358316, 'depth': 12, 'random_strength': 2.6015638139495264e-07, 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise', 'od_type': 'Iter', 'od_wait': 42}. Best is trial 27 with value: 0.4966336815104927.


...

Trial 36 finished with value: 0.6362447295890847 and parameters: {'learning_rate': 0.0010480764156439555, 'l2_leaf_reg': 66.13306579867303, 'colsample_bylevel': 0.0276650279619753, 'depth': 11, 'random_strength': 6.787314439854602e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 50, 'bagging_temperature': 0.07464435123501412}. Best is trial 35 with value: 0.6365822214704931.


[I 2023-04-06 17:26:25,434] Trial 37 finished with value: 0.6368050104572306 and parameters: {'learning_rate': 0.0011652232004604518, 'l2_leaf_reg': 81.3907170361274, 'colsample_bylevel': 0.024089164985701322, 'depth': 11, 'random_strength': 4.224658318264939e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 45, 'bagging_temperature': 0.2614670921933523}. Best is trial 37 with value: 0.6368050104572306.


Trial 37 finished with value: 0.6368050104572306 and parameters: {'learning_rate': 0.0011652232004604518, 'l2_leaf_reg': 81.3907170361274, 'colsample_bylevel': 0.024089164985701322, 'depth': 11, 'random_strength': 4.224658318264939e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 45, 'bagging_temperature': 0.2614670921933523}. Best is trial 37 with value: 0.6368050104572306.


[I 2023-04-06 17:26:37,382] Trial 38 finished with value: 0.6440391395231012 and parameters: {'learning_rate': 0.0010186005490146904, 'l2_leaf_reg': 85.1403266884871, 'colsample_bylevel': 0.025889525427018403, 'depth': 11, 'random_strength': 2.499376073859301e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 45, 'bagging_temperature': 0.5475073039000595}. Best is trial 38 with value: 0.6440391395231012.


Trial 38 finished with value: 0.6440391395231012 and parameters: {'learning_rate': 0.0010186005490146904, 'l2_leaf_reg': 85.1403266884871, 'colsample_bylevel': 0.025889525427018403, 'depth': 11, 'random_strength': 2.499376073859301e-08, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Lossguide', 'od_type': 'Iter', 'od_wait': 45, 'bagging_temperature': 0.5475073039000595}. Best is trial 38 with value: 0.6440391395231012.


In [ ]:
optuna.visualization.plot_intermediate_values(study)

In [12]:
optuna.visualization.plot_optimization_history(study)

In [13]:
optuna.visualization.plot_param_importances(study)

In [131]:
optuna.visualization.plot_slice(study)

In [132]:
optuna.visualization.plot_parallel_coordinate(study, ["colsample_bylevel", "learning_rate"])

In [133]:
optuna.visualization.plot_contour(study, ["colsample_bylevel", "learning_rate"])

In [54]:
x = study.best_params

In [58]:
y = {"test": 1, "bootstrap_type": "Regan"}

In [59]:
x.update(y)
x

{'bagging_temperature': 5.8195550823314885,
 'bootstrap_type': 'Regan',
 'colsample_bylevel': 0.033486344069417806,
 'depth': 8,
 'grow_policy': 'SymmetricTree',
 'l2_leaf_reg': 0.00018823979324111533,
 'learning_rate': 0.08600382937333881,
 'od_type': 'Iter',
 'od_wait': 47,
 'random_strength': 0.053068162215817234,
 'test': 1}

In [70]:
m = cb.CatBoostClassifier(objective="MultiClassOneVsAll", verbose=0)
m.fit(X_test, y_test)
m.predict_proba(X_test)

array([[0.00196355, 0.99803645],
       [0.00221492, 0.99778508],
       [0.03721321, 0.96278679],
       [0.97859238, 0.02140762],
       [0.00281328, 0.99718672],
       [0.00335827, 0.99664173],
       [0.9964222 , 0.0035778 ],
       [0.97925075, 0.02074925],
       [0.00216228, 0.99783772],
       [0.9939794 , 0.0060206 ],
       [0.9947634 , 0.0052366 ],
       [0.84320751, 0.15679249],
       [0.00700223, 0.99299777],
       [0.02705514, 0.97294486],
       [0.00162448, 0.99837552],
       [0.00531296, 0.99468704],
       [0.98640895, 0.01359105],
       [0.99685856, 0.00314144],
       [0.00302852, 0.99697148],
       [0.00644896, 0.99355104],
       [0.00589376, 0.99410624],
       [0.00318716, 0.99681284],
       [0.00426834, 0.99573166],
       [0.98812327, 0.01187673],
       [0.00709457, 0.99290543],
       [0.99501073, 0.00498927],
       [0.99623123, 0.00376877],
       [0.99438428, 0.00561572],
       [0.90023372, 0.09976628],
       [0.00217423, 0.99782577],
       [0.

In [8]:
m = cb.CatBoostClassifier(objective="Logloss", verbose=1, eval_metric="Logloss")
m.fit(X_test, y_test, eval_set=[(X_test, y_test)])
pred_proba = m.predict_proba(X_test)
pred_proba

Learning rate set to 0.033485
0:	learn: 0.6886507	test: 0.6886564	best: 0.6886564 (0)	total: 19.4ms	remaining: 19.4s
1:	learn: 0.6845698	test: 0.6845768	best: 0.6845768 (1)	total: 32.9ms	remaining: 16.4s
2:	learn: 0.6798040	test: 0.6798150	best: 0.6798150 (2)	total: 44.7ms	remaining: 14.9s
3:	learn: 0.6749502	test: 0.6749643	best: 0.6749643 (3)	total: 56.1ms	remaining: 14s
4:	learn: 0.6714234	test: 0.6714398	best: 0.6714398 (4)	total: 68.7ms	remaining: 13.7s
5:	learn: 0.6665727	test: 0.6665932	best: 0.6665932 (5)	total: 80.3ms	remaining: 13.3s
6:	learn: 0.6612063	test: 0.6612304	best: 0.6612304 (6)	total: 91.5ms	remaining: 13s
7:	learn: 0.6559053	test: 0.6559336	best: 0.6559336 (7)	total: 102ms	remaining: 12.7s
8:	learn: 0.6509487	test: 0.6509806	best: 0.6509806 (8)	total: 113ms	remaining: 12.5s
9:	learn: 0.6465319	test: 0.6465660	best: 0.6465660 (9)	total: 125ms	remaining: 12.3s
10:	learn: 0.6425506	test: 0.6425878	best: 0.6425878 (10)	total: 136ms	remaining: 12.2s
11:	learn: 0.637591

99:	learn: 0.3970974	test: 0.3971094	best: 0.3971094 (99)	total: 1.23s	remaining: 11s
100:	learn: 0.3952738	test: 0.3952857	best: 0.3952857 (100)	total: 1.24s	remaining: 11s
101:	learn: 0.3937077	test: 0.3937190	best: 0.3937190 (101)	total: 1.25s	remaining: 11s
102:	learn: 0.3925304	test: 0.3925416	best: 0.3925416 (102)	total: 1.26s	remaining: 11s
103:	learn: 0.3904046	test: 0.3904157	best: 0.3904157 (103)	total: 1.27s	remaining: 11s
104:	learn: 0.3888264	test: 0.3888375	best: 0.3888375 (104)	total: 1.28s	remaining: 11s
105:	learn: 0.3877909	test: 0.3878018	best: 0.3878018 (105)	total: 1.3s	remaining: 10.9s
106:	learn: 0.3861673	test: 0.3861785	best: 0.3861785 (106)	total: 1.31s	remaining: 10.9s
107:	learn: 0.3841847	test: 0.3841959	best: 0.3841959 (107)	total: 1.32s	remaining: 10.9s
108:	learn: 0.3820588	test: 0.3820698	best: 0.3820698 (108)	total: 1.33s	remaining: 10.9s
109:	learn: 0.3803696	test: 0.3803808	best: 0.3803808 (109)	total: 1.34s	remaining: 10.9s
110:	learn: 0.3790870	tes

202:	learn: 0.2544265	test: 0.2544480	best: 0.2544480 (202)	total: 2.45s	remaining: 9.61s
203:	learn: 0.2531034	test: 0.2531250	best: 0.2531250 (203)	total: 2.46s	remaining: 9.6s
204:	learn: 0.2518604	test: 0.2518822	best: 0.2518822 (204)	total: 2.48s	remaining: 9.61s
205:	learn: 0.2501952	test: 0.2502178	best: 0.2502178 (205)	total: 2.49s	remaining: 9.6s
206:	learn: 0.2486091	test: 0.2486318	best: 0.2486318 (206)	total: 2.5s	remaining: 9.59s
207:	learn: 0.2472831	test: 0.2473061	best: 0.2473061 (207)	total: 2.51s	remaining: 9.57s
208:	learn: 0.2460847	test: 0.2461077	best: 0.2461077 (208)	total: 2.52s	remaining: 9.55s
209:	learn: 0.2448980	test: 0.2449211	best: 0.2449211 (209)	total: 2.54s	remaining: 9.54s
210:	learn: 0.2436857	test: 0.2437093	best: 0.2437093 (210)	total: 2.55s	remaining: 9.53s
211:	learn: 0.2422292	test: 0.2422531	best: 0.2422531 (211)	total: 2.56s	remaining: 9.52s
212:	learn: 0.2408227	test: 0.2408469	best: 0.2408469 (212)	total: 2.57s	remaining: 9.5s
213:	learn: 0.

306:	learn: 0.1361446	test: 0.1361876	best: 0.1361876 (306)	total: 3.68s	remaining: 8.3s
307:	learn: 0.1351363	test: 0.1351791	best: 0.1351791 (307)	total: 3.69s	remaining: 8.29s
308:	learn: 0.1343041	test: 0.1343470	best: 0.1343470 (308)	total: 3.7s	remaining: 8.28s
309:	learn: 0.1331545	test: 0.1331979	best: 0.1331979 (309)	total: 3.71s	remaining: 8.27s
310:	learn: 0.1323110	test: 0.1323545	best: 0.1323545 (310)	total: 3.73s	remaining: 8.26s
311:	learn: 0.1316871	test: 0.1317301	best: 0.1317301 (311)	total: 3.74s	remaining: 8.25s
312:	learn: 0.1308360	test: 0.1308784	best: 0.1308784 (312)	total: 3.75s	remaining: 8.23s
313:	learn: 0.1301079	test: 0.1301507	best: 0.1301507 (313)	total: 3.76s	remaining: 8.22s
314:	learn: 0.1292389	test: 0.1292816	best: 0.1292816 (314)	total: 3.77s	remaining: 8.21s
315:	learn: 0.1282826	test: 0.1283250	best: 0.1283250 (315)	total: 3.79s	remaining: 8.2s
316:	learn: 0.1272518	test: 0.1272940	best: 0.1272940 (316)	total: 3.8s	remaining: 8.19s
317:	learn: 0.

410:	learn: 0.0742662	test: 0.0743086	best: 0.0743086 (410)	total: 4.91s	remaining: 7.03s
411:	learn: 0.0737918	test: 0.0738340	best: 0.0738340 (411)	total: 4.92s	remaining: 7.02s
412:	learn: 0.0734640	test: 0.0735063	best: 0.0735063 (412)	total: 4.93s	remaining: 7.01s
413:	learn: 0.0730952	test: 0.0731370	best: 0.0731370 (413)	total: 4.94s	remaining: 6.99s
414:	learn: 0.0727756	test: 0.0728170	best: 0.0728170 (414)	total: 4.95s	remaining: 6.98s
415:	learn: 0.0724313	test: 0.0724726	best: 0.0724726 (415)	total: 4.96s	remaining: 6.97s
416:	learn: 0.0719756	test: 0.0720169	best: 0.0720169 (416)	total: 4.98s	remaining: 6.96s
417:	learn: 0.0716223	test: 0.0716636	best: 0.0716636 (417)	total: 4.99s	remaining: 6.95s
418:	learn: 0.0712042	test: 0.0712451	best: 0.0712451 (418)	total: 5s	remaining: 6.93s
419:	learn: 0.0708466	test: 0.0708875	best: 0.0708875 (419)	total: 5.01s	remaining: 6.92s
420:	learn: 0.0704570	test: 0.0704981	best: 0.0704981 (420)	total: 5.02s	remaining: 6.91s
421:	learn: 0

...

619:	learn: 0.0283212	test: 0.0283469	best: 0.0283469 (619)	total: 7.36s	remaining: 4.51s
620:	learn: 0.0282099	test: 0.0282356	best: 0.0282356 (620)	total: 7.37s	remaining: 4.5s
621:	learn: 0.0280950	test: 0.0281206	best: 0.0281206 (621)	total: 7.39s	remaining: 4.49s
622:	learn: 0.0279737	test: 0.0279990	best: 0.0279990 (622)	total: 7.4s	remaining: 4.48s
623:	learn: 0.0278721	test: 0.0278974	best: 0.0278974 (623)	total: 7.41s	remaining: 4.47s
624:	learn: 0.0277588	test: 0.0277840	best: 0.0277840 (624)	total: 7.42s	remaining: 4.45s
625:	learn: 0.0276162	test: 0.0276413	best: 0.0276413 (625)	total: 7.43s	remaining: 4.44s
626:	learn: 0.0274986	test: 0.0275239	best: 0.0275239 (626)	total: 7.45s	remaining: 4.43s
627:	learn: 0.0274122	test: 0.0274375	best: 0.0274375 (627)	total: 7.46s	remaining: 4.42s
628:	learn: 0.0272636	test: 0.0272887	best: 0.0272887 (628)	total: 7.47s	remaining: 4.41s
629:	learn: 0.0271444	test: 0.0271693	best: 0.0271693 (629)	total: 7.48s	remaining: 4.39s
630:	learn: 

722:	learn: 0.0191147	test: 0.0191332	best: 0.0191332 (722)	total: 8.59s	remaining: 3.29s
723:	learn: 0.0190566	test: 0.0190752	best: 0.0190752 (723)	total: 8.6s	remaining: 3.28s
724:	learn: 0.0189887	test: 0.0190072	best: 0.0190072 (724)	total: 8.62s	remaining: 3.27s
725:	learn: 0.0189293	test: 0.0189478	best: 0.0189478 (725)	total: 8.63s	remaining: 3.25s
726:	learn: 0.0188534	test: 0.0188719	best: 0.0188719 (726)	total: 8.64s	remaining: 3.24s
727:	learn: 0.0187800	test: 0.0187986	best: 0.0187986 (727)	total: 8.65s	remaining: 3.23s
728:	learn: 0.0187079	test: 0.0187266	best: 0.0187266 (728)	total: 8.66s	remaining: 3.22s
729:	learn: 0.0186433	test: 0.0186617	best: 0.0186617 (729)	total: 8.67s	remaining: 3.21s
730:	learn: 0.0185851	test: 0.0186036	best: 0.0186036 (730)	total: 8.69s	remaining: 3.2s
731:	learn: 0.0185236	test: 0.0185421	best: 0.0185421 (731)	total: 8.7s	remaining: 3.18s
732:	learn: 0.0184634	test: 0.0184819	best: 0.0184819 (732)	total: 8.71s	remaining: 3.17s
733:	learn: 0

826:	learn: 0.0137527	test: 0.0137667	best: 0.0137667 (826)	total: 9.82s	remaining: 2.05s
827:	learn: 0.0137130	test: 0.0137269	best: 0.0137269 (827)	total: 9.83s	remaining: 2.04s
828:	learn: 0.0136679	test: 0.0136820	best: 0.0136820 (828)	total: 9.84s	remaining: 2.03s
829:	learn: 0.0136195	test: 0.0136332	best: 0.0136332 (829)	total: 9.85s	remaining: 2.02s
830:	learn: 0.0135679	test: 0.0135815	best: 0.0135815 (830)	total: 9.87s	remaining: 2.01s
831:	learn: 0.0135401	test: 0.0135537	best: 0.0135537 (831)	total: 9.88s	remaining: 1.99s
832:	learn: 0.0134949	test: 0.0135085	best: 0.0135085 (832)	total: 9.89s	remaining: 1.98s
833:	learn: 0.0134515	test: 0.0134651	best: 0.0134651 (833)	total: 9.9s	remaining: 1.97s
834:	learn: 0.0134063	test: 0.0134200	best: 0.0134200 (834)	total: 9.91s	remaining: 1.96s
835:	learn: 0.0133538	test: 0.0133674	best: 0.0133674 (835)	total: 9.93s	remaining: 1.95s
836:	learn: 0.0133103	test: 0.0133239	best: 0.0133239 (836)	total: 9.94s	remaining: 1.94s
837:	learn:

931:	learn: 0.0103037	test: 0.0103147	best: 0.0103147 (931)	total: 11.1s	remaining: 807ms
932:	learn: 0.0102816	test: 0.0102925	best: 0.0102925 (932)	total: 11.1s	remaining: 795ms
933:	learn: 0.0102612	test: 0.0102721	best: 0.0102721 (933)	total: 11.1s	remaining: 783ms
934:	learn: 0.0102359	test: 0.0102468	best: 0.0102468 (934)	total: 11.1s	remaining: 771ms
935:	learn: 0.0102122	test: 0.0102230	best: 0.0102230 (935)	total: 11.1s	remaining: 759ms
936:	learn: 0.0101869	test: 0.0101977	best: 0.0101977 (936)	total: 11.1s	remaining: 747ms
937:	learn: 0.0101645	test: 0.0101753	best: 0.0101753 (937)	total: 11.1s	remaining: 735ms
938:	learn: 0.0101352	test: 0.0101460	best: 0.0101460 (938)	total: 11.1s	remaining: 724ms
939:	learn: 0.0101042	test: 0.0101151	best: 0.0101151 (939)	total: 11.1s	remaining: 712ms
940:	learn: 0.0100778	test: 0.0100886	best: 0.0100886 (940)	total: 11.2s	remaining: 700ms
941:	learn: 0.0100591	test: 0.0100699	best: 0.0100699 (941)	total: 11.2s	remaining: 688ms
942:	learn

array([[0.01268483, 0.98731517],
       [0.00832655, 0.99167345],
       [0.01071938, 0.98928062],
       ...,
       [0.01411076, 0.98588924],
       [0.01761985, 0.98238015],
       [0.99343418, 0.00656582]])

In [14]:
log_loss(y_test, pred_proba)

0.10856780010472185

In [12]:
neg_log_loss = make_scorer(score_func=log_loss, greater_is_better=False, needs_proba=True)
f

make_scorer(log_loss, greater_is_better=False, needs_proba=True)

In [13]:
f(m, X_test, y_test)

-0.10856780010472185

In [7]:
study.trials[0].datetime_start

datetime.datetime(2023, 4, 6, 16, 55, 51, 83959)

In [12]:
study.trials[0].value

-0.45003056705495253

In [19]:
study.trials[0].params

{'bootstrap_type': 'Bernoulli',
 'colsample_bylevel': 0.06298201423587532,
 'depth': 1,
 'grow_policy': 'Depthwise',
 'l2_leaf_reg': 1.533961499604723e-07,
 'learning_rate': 0.2294443901890887,
 'od_type': 'Iter',
 'od_wait': 28,
 'random_strength': 7.80784562417798e-08,
 'subsample': 0.4850923023026483}

In [18]:
old_trials = study.get_trials(deepcopy=False)
correct_trials = [
    optuna.trial.create_trial(
        state=optuna.trial.TrialState.COMPLETE,
        params=trial.params,
        distributions=trial.distributions,
        value=trial.value
    ) for trial in old_trials]
new_study = optuna.create_study()
new_study.add_trials(correct_trials)

ValueError: `value` is supposed to be set for a complete trial.

In [ ]:
old_trials = study.get_trials(deepcopy=False)
old_trials

In [12]:
import datetime

In [13]:
datetime.datetime.

datetime.datetime

In [23]:
study.trials[1]

FrozenTrial(number=1, state=TrialState.COMPLETE, values=[-0.6306997787474536], datetime_start=datetime.datetime(2023, 4, 6, 16, 55, 52, 297626), datetime_complete=datetime.datetime(2023, 4, 6, 16, 55, 57, 85356), params={'bootstrap_type': 'Bernoulli', 'colsample_bylevel': 0.011589924229601182, 'depth': 10, 'grow_policy': 'Depthwise', 'l2_leaf_reg': 1.2208460438070453e-07, 'learning_rate': 0.011996341480658446, 'od_type': 'IncToDec', 'od_wait': 10, 'random_strength': 0.00026098261562786524, 'subsample': 0.20131755529216386}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.6931759164812252, 1: 0.6927834715637176, 2: 0.6920532670673256, 3: 0.6920988929155047, 4: 0.6912474474647206, 5: 0.6908674340739578, 6: 0.6906137746827155, 7: 0.6899512512444698, 8: 0.6895564088730054, 9: 0.6897786803638325, 10: 0.6897383347714497, 11: 0.6893570898028709, 12: 0.6897654689867644, 13: 0.68990459554715, 14: 0.6893726634439428, 15: 0.6895692757229482, 16: 0.689468312040408, 17: 0.6894220778263765

In [18]:
num_trials = study.trials[-1].number
num_trials_to_keep = 20

In [19]:
[t for t in study.trials if t.number > num_trials - num_trials_to_keep]

[FrozenTrial(number=39, state=TrialState.COMPLETE, values=[-0.6511535966073816], datetime_start=datetime.datetime(2023, 4, 11, 9, 28, 8, 218626), datetime_complete=datetime.datetime(2023, 4, 11, 9, 28, 37, 132832), params={'bootstrap_type': 'Bayesian', 'colsample_bylevel': 0.024582970703969854, 'depth': 11, 'grow_policy': 'Lossguide', 'l2_leaf_reg': 93.70655010457759, 'learning_rate': 0.0010671445753259938, 'od_type': 'Iter', 'od_wait': 45, 'random_strength': 3.415251558759658e-08}, user_attrs={'mean_fit_time': 9.52130619684855, 'mean_score_time': 0.08929308255513509, 'mean_test_score': -0.6511535966073816, 'split0_test_score': -0.6507956278749639, 'split1_test_score': -0.6516396159526993, 'split2_test_score': -0.6510255459944815, 'std_fit_time': 0.07619737102926176, 'std_score_time': 0.004746037688996099, 'std_test_score': 0.00035625523475724553}, system_attrs={}, intermediate_values={}, distributions={'bootstrap_type': CategoricalDistribution(choices=('Bayesian', 'Bernoulli', 'MVS'))

In [33]:
from optuna.trial import TrialState

num_trials = study.trials[-1].number
num_trials_to_keep = 20

trials_to_keep = [
    optuna.trial.create_trial(
        state=TrialState.COMPLETE,
        params=t.params,
        user_attrs=t.user_attrs,
        system_attrs=t.system_attrs,
        intermediate_values=t.intermediate_values,
        distributions=t.distributions,
        value=t.value
    ) for t in study.trials if t.number > num_trials - num_trials_to_keep]

# Delete study before recreating
optuna.delete_study(study_name=study_name, storage=storage_name)

new_study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), 
    direction="minimize",
    study_name=study_name, 
    storage=storage_name,
    load_if_exists=True)
new_study.add_trials(trials_to_keep)

[I 2023-04-13 21:36:02,429] A new study created in RDB with name: Test10


A new study created in RDB with name: Test10


In [34]:
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), 
    direction="minimize",
    study_name=study_name, 
    storage=storage_name,
    load_if_exists=True
    )

[I 2023-04-13 21:36:19,260] Using an existing study with name 'Test10' instead of creating a new one.


Using an existing study with name 'Test10' instead of creating a new one.


In [37]:
len(study.trials)

20

In [ ]:
study.